In [3]:
import numpy as np
import math
import cv2
import sys
import scipy.io as sio
import matplotlib.pyplot as plt
import os
import h5py

In [4]:
def load_wiki(wiki_path, num_data = None, target_size = (224,224)):
    mat_path = wiki_path + 'wiki_with_age.mat'
    
    data = sio.loadmat(mat_path)
    wiki_data = data['wiki'][0][0]
   
    full_num_data = len(wiki_data[6][0])
    if num_data is None or num_data > full_num_data :
        num_data = full_num_data
        
    
    full_X_data = np.zeros([num_data, 224, 224, 3],dtype = "uint8")
    full_y_data = np.zeros([num_data],dtype = "uint8")
    data_count = 0
    #len(wiki_data[6][0])
    for i in range(num_data):

        face_score =wiki_data[6][0][i]
        if face_score != float("-inf"):
            full_path = wiki_path + wiki_data[2][0][i][0]
            img = cv2.imread(full_path)
            age = wiki_data[8][0][i]
            date_of_birth = wiki_data[0][0][i]  #下面的657438是出生于1800年的Matlab serial date number
            gender = wiki_data[3][0][i]
            if img is not None and gender == gender and date_of_birth > 657438 and age >= 0 and age <= 100:
                face_loc = wiki_data[5][0][i][0]
                face_loc = face_loc.astype("int32")
                roi_img = img[face_loc[1]:face_loc[3], face_loc[0]:face_loc[2]]
                face_img = cv2.resize(roi_img, target_size)
                
                full_X_data[data_count] = face_img                
                full_y_data[data_count] = gender * 100 + age #一共分为200类，0-99是女性的1-100岁，100-199是男性的1-100岁
                
                data_count += 1
    X_data = full_X_data[:data_count]
    y_data = full_y_data[:data_count]
    
    return X_data, y_data

In [5]:
def get_wiki_data(X_data, y_data, num_training=49000, num_validation=1000, num_test=1000):
   
    X_train = X_data[:-(num_validation + num_test)]
    y_train = y_data[:-(num_validation + num_test)]
    X_val = X_data[X_train.shape[0] : -num_test]
    y_val = y_data[X_train.shape[0] : -num_test]
    X_test = X_data[-num_test:]
    y_test = y_data[-num_test:]
    """
    if X_data.shape[0] > num_training + num_validation + num_test:
        # Subsample the data
        mask = list(range(num_training + num_validation ,  num_training + num_validation + num_test))
        X_test = X_data[mask]
        y_test = y_data[mask]
        mask = list(range(num_training, num_training + num_validation))
        X_val = X_data[mask]
        y_val = y_data[mask]
        mask = list(range(num_training))
        X_train = X_data[mask]
        y_train = y_data[mask]
    """
    # Normalize the data: subtract the mean image
    """if subtract_mean:
        mean_image = np.mean(X_data, axis=0)
        X_train -= mean_image
        X_val -= mean_image
        X_test -= mean_image"""
    # Transpose so that channels come first
    X_train = X_train.transpose(0, 3, 1, 2).copy()
    X_val = X_val.transpose(0, 3, 1, 2).copy()
    X_test = X_test.transpose(0, 3, 1, 2).copy()

    # Package data into a dictionary
    return {
      'X_train': X_train, 'y_train': y_train,
      'X_val': X_val, 'y_val': y_val,
      'X_test': X_test, 'y_test': y_test,
    }
    

In [10]:

wiki_path = ""
if sys.platform == "linux" :
    wiki_path = "/devdata/wiki/"
else:
    wiki_path = "G:\\MachineLearning\\wiki\\wiki\\"

mat_path = wiki_path + 'wiki_with_age.mat'

# Create a new file
f = None
X_data = None
y_data = None
#wiki_crop_dataset = f.create_dataset('wiki_cropface_data', dtype = "int32")
if not os.path.exists('/devdata/wiki_cropface_data.h5'):
    f = h5py.File('/devdata/wiki_cropface_data.h5', "r")
    wiki_cropface_group = f["wiki_cropface_group"]
    X_data = np.array(wiki_cropface_group["X_data"][:])
    y_data = np.array(wiki_cropface_group["y_data"][:])
else:
    #f = h5py.File('/devdata/wiki_cropface_data.h5',"w")
    X_data, y_data  = load_wiki(wiki_path, 100)
    #wiki_cropface_group = f.create_group("wiki_cropface_group")
    #wiki_cropface_group.create_dataset('X_data', dtype = "uint8", data = X_data)
    #wiki_cropface_group.create_dataset('y_data', dtype = "uint8", data = y_data)
    
print(X_data[0].shape)
print(X_data[0].dtype)
wiki_cropface_dataset = get_wiki_data(X_data, y_data, num_training=49000, num_validation=10, num_test=0)
#f.close()

"""
test_img = X_data[0]
cv2.imshow("test_img", test_img)
cv2.waitKey()
cv2.destroyAllWindows()"""


(224, 224, 3)
uint8


'\ntest_img = X_data[0]\ncv2.imshow("test_img", test_img)\ncv2.waitKey()\ncv2.destroyAllWindows()'

In [7]:
"""temp = cv2.imread("C:\\Users\\Z\\Desktop\\2006361_1907-12-25_1945.gif")
print(temp.shape)
cv2.imshow(temp)
cv2.waitKey()
cv2.destroyAllWindows()"""
import numpy
x = numpy.arange(100)
print(x)
print(x[:None])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
